# 점별 상호 정보량(PMI)  

단순히 동시에 등장했다고 해서 반드시 연관성이 있는가? 이를 고집한다면 the 같은 녀석들이 유사도가 높다고 나올 것이다. PMI 기준으로 유사도를 재판단해야 한다.

$$ PMI(x, y) = log_2(\frac{P(x, y)}{P(x)P(y)}) $$ 

다만 $$ log_2{0} = -\infin $$ 이므로 max(0)로 한 번 감싼 양의 상호 정보량을 활용한다.

$$ PPMI(x, y) = max(0, PMI(x, y)) $$ 

In [7]:
import sys
import numpy as np
sys.path.append("..")
from common.util import preprocess, create_co_matrix, cos_similarity, ppmi

In [5]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)
W

array([[0.       , 1.8073549, 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [1.8073549, 0.       , 0.8073549, 0.       , 0.8073549, 0.8073549,
        0.       ],
       [0.       , 0.8073549, 0.       , 1.8073549, 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.8073549, 0.       , 1.8073549, 0.       ,
        0.       ],
       [0.       , 0.8073549, 0.       , 1.8073549, 0.       , 0.       ,
        0.       ],
       [0.       , 0.8073549, 0.       , 0.       , 0.       , 0.       ,
        2.807355 ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 2.807355 ,
        0.       ]], dtype=float32)

In [8]:
np.set_printoptions(precision=3)
print("동시 발생 행렬")
print(C)
print('-'*50)
print("PPMI 행렬")
print(W)

동시 발생 행렬
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
PPMI 행렬
[[0.    1.807 0.    0.    0.    0.    0.   ]
 [1.807 0.    0.807 0.    0.807 0.807 0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.    1.807 0.    1.807 0.    0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.807 0.    0.    0.    0.    2.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


PPMI 행렬로 좀 더 정확한 단어 분포 매트릭스를 얻었지만 0이 많은 sparse matrix라서 차원 축소를 하여 dense하게 만들어야 함  

우리는 여기서 특이값분해 (SVD, singular value decomposition)을 해보려고 함  

SVD는 임의의 행렬을 세 행렬(U, S, V)의 곱으로 분해하며, 수식으로는 다음과 같음.

$$ X = USV^{T} $$  

여기서 U, V를 직교 행렬이고, 그 열벡터는 서로 직교함  
S는 대각 행렬임 (대각선 빼고 다 0인 행렬)  

In [11]:
U, S, V = np.linalg.svd(W) # U가 차원 축소된 embedding vector임

In [12]:
# 단어 id가 0인 단어 벡터를 살펴보자면
print(C[0])
print(W[0])
print(U[0])

[0 1 0 0 0 0 0]
[0.    1.807 0.    0.    0.    0.    0.   ]
[-3.409e-01 -1.110e-16 -3.886e-16 -1.205e-01  0.000e+00  9.323e-01
  2.226e-16]
